In [142]:
!pip install pytorch-tabnet

In [143]:
from pytorch_tabnet.pretraining import TabNetPretrainer
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import pandas as pd
#df = pd.read_csv("train.csv")
#詳しくはここhttps://github.com/dreamquark-ai/tabnet/tree/develop

In [144]:
from sklearn.datasets import fetch_california_housing
data = fetch_california_housing()

In [145]:
df = np.concatenate([data.data, data.target.reshape(20640,1)], axis = 1)

In [146]:
X_train, X_valid, X_test = df[:int(0.8*len(df))], df[int(0.8*len(df)):int(0.9*len(df))],df[int(0.9*len(df)):]

In [147]:
unsupervised_model = TabNetPretrainer(
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=2e-2),
    mask_type='entmax' # "sparsemax"
)

unsupervised_model.fit(
    X_train=X_train,
    eval_set=[X_valid],
    pretraining_ratio=0.8,
    max_epochs=500,
    patience=35,
)

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 418.27955| val_0_unsup_loss_numpy: 3633.772705078125|  0:00:02s
epoch 1  | loss: 290.70801| val_0_unsup_loss_numpy: 2458.705810546875|  0:00:05s
epoch 2  | loss: 138.6288| val_0_unsup_loss_numpy: 2745.99755859375|  0:00:07s
epoch 3  | loss: 26.27022| val_0_unsup_loss_numpy: 1885.794677734375|  0:00:10s
epoch 4  | loss: 3.8508  | val_0_unsup_loss_numpy: 1768.054931640625|  0:00:13s
epoch 5  | loss: 2.18467 | val_0_unsup_loss_numpy: 987.9741821289062|  0:00:15s
epoch 6  | loss: 2.03046 | val_0_unsup_loss_numpy: 376.2887878417969|  0:00:18s
epoch 7  | loss: 1.62305 | val_0_unsup_loss_numpy: 1765.5413818359375|  0:00:20s
epoch 8  | loss: 1.43801 | val_0_unsup_loss_numpy: 582.4559936523438|  0:00:22s
epoch 9  | loss: 1.30222 | val_0_unsup_loss_numpy: 440.5889892578125|  0:00:24s
epoch 10 | loss: 1.36428 | val_0_unsup_loss_numpy: 377.00537109375|  0:00:26s
epoch 11 | loss: 1.29948 | val_0_unsup_loss_numpy: 446.3124694824219|  0:00:27s
epoch 12 | loss: 1.2297  | val_0_unsup_l

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [148]:
unsupervised_encoder = unsupervised_model.network.encoder
unsupervised_decoder = unsupervised_model.network.decoder

In [149]:
batch_data = torch.tensor(X_train).to(torch.float32).to("cuda")

In [150]:
def create_noise(col, row, device = "cuda"):
  noise = [torch.tensor(np.random.rand(col, row)).to(torch.float32).to(device) for _ in range(3)]
  return noise
noise = create_noise(3, 8,)

In [151]:
class Discriminator(nn.Module):
  def __init__(self, encoder, features):
    super().__init__()
    for param in encoder.parameters():
      param.requires_grad = False
    self.encoder = encoder
    self.features = features #decoderに通した後の出力のrow数を意味する
    self.ffn = nn.Linear(self.features*3, self.features, bias = False)
    self.relu = nn.ReLU()
    self.linear = nn.Linear(self.features, 2, bias = False)
    self.sigmoid = nn.Sigmoid()
  def forward(self, table_data):
    extracted_data, _ = self.encoder(table_data)
    x = extracted_data[0]
    for ext_data in extracted_data[1:]:
      x = torch.concatenate([x, ext_data], dim = 1)
    x = self.ffn(x)
    x = self.relu(x)
    x = self.linear(x)
    x = self.sigmoid(x)
    return x

In [152]:
disc = Discriminator(unsupervised_encoder, 8).to("cuda")
gene = unsupervised_decoder.to("cuda")
optimizerD = optim.Adam(disc.parameters(), lr = 1e-4)
optimizerG = optim.Adam(gene.parameters(), lr = 1e-4)
col = 16512
row = 8
def trainTGAN(optimizerD, optimizerG, real_data, device = "cuda"):
  #まず識別器のモデルを改善した後に生成器のモデルを改善する
    criterion = nn.BCELoss()
    optimizerD.zero_grad()
    batch_size = real_data.size(0)
    real_proba = disc(real_data) #データの形状が合わない時はforward内部で入力ように整形変更することを想定して制作。
    real_label = torch.ones_like(real_proba).to(device)
    real_label[:,0] = 0
    D_loss_real = criterion(real_proba, real_label)
    input_z = create_noise(col, row,)
    fake_data = gene(input_z)
    fake_proba = disc(fake_data)
    fake_label = torch.ones_like(fake_proba).to(device)
    fake_label[:,1] = 0
    D_loss_fake = criterion(fake_proba,fake_label)
    D_loss = D_loss_fake + D_loss_real
    D_loss.backward()
    optimizerD.step()
    #D_lossのところでGenerate Modelのパラメータの勾配も計算されている為、Generate Modelに関する勾配はここで初期化を行う
    optimizerG.zero_grad()
    input_z = create_noise(col, row,)
    fake_data = gene(input_z)
    fake_proba = disc(fake_data)
    real_label = torch.ones_like(fake_proba).to(device)
    real_label[:,0] = 0 #ラベルが1になるように(騙すように)訓練するので、ラベルは1を予測するように作る
    G_loss = criterion(fake_proba,real_label)
    G_loss.backward()
    optimizerG.step()
    return D_loss.detach().item(), G_loss.detach().item()

In [153]:
epochs = 750
D_losses = []
G_losses = []
from tqdm import tqdm
for epoch in tqdm(range(epochs)):
  D_loss, G_loss = trainTGAN(optimizerD, optimizerG, batch_data, device = "cuda")
  if epoch % 10 == 0:
        print(f"Discriminator Loss: {D_loss},Generater Loss: {G_loss}")
print(f"Discriminator Loss: {D_loss},Generater Loss: {G_loss}")

  0%|          | 1/750 [00:00<08:22,  1.49it/s]

Discriminator Loss: 1.3839466571807861,Generater Loss: 0.7142834067344666


  1%|▏         | 11/750 [00:07<08:02,  1.53it/s]

Discriminator Loss: 1.3837580680847168,Generater Loss: 0.7140856385231018


  3%|▎         | 21/750 [00:14<08:23,  1.45it/s]

...

Discriminator Loss: 1.3841367959976196,Generater Loss: 0.6831267476081848


 99%|█████████▉| 741/750 [09:38<00:06,  1.47it/s]

Discriminator Loss: 1.3833162784576416,Generater Loss: 0.6853542923927307


100%|██████████| 750/750 [09:46<00:00,  1.28it/s]

Discriminator Loss: 1.3823182582855225,Generater Loss: 0.6878888010978699


In [154]:
import lightgbm as lgb
lgb_rg = lgb.LGBMRegressor(max_depth = 100, n_estimators = 100)

In [155]:
lgb_rg.fit(X_train[:,:-1], X_train[:,-1])

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001979 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1837
[LightGBM] [Info] Number of data points in the train set: 16512, number of used features: 8
[LightGBM] [Info] Start training from score 2.020670


LGBMRegressor(max_depth=100)

In [156]:
pred = lgb_rg.predict(X_test[:,:-1])

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


In [157]:
(pred - X_test[:,-1]).std()
#元のデータでの予測

0.4670865970792997

In [158]:
input_z = create_noise(col, row,)
aug_data = gene(input_z).detach().cpu().numpy()

In [159]:
lgb_rg = lgb.LGBMRegressor(max_depth = 100, n_estimators = 100)
lgb_rg.fit(aug_data[:,:-1], aug_data[:,-1])

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001940 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2040
[LightGBM] [Info] Number of data points in the train set: 16512, number of used features: 8
[LightGBM] [Info] Start training from score 3.403901


LGBMRegressor(max_depth=100)

In [160]:
pred = lgb_rg.predict(X_test[:,:-1])
(pred - X_test[:,-1]).std()
#増殖したデータのみでの予測

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


0.6502803279115924

In [161]:
#さらに増やす
input_z = create_noise(30000, row,)
aug_data = gene(input_z).detach().cpu().numpy()
lgb_rg = lgb.LGBMRegressor(max_depth = 100, n_estimators = 100)
lgb_rg.fit(aug_data[:,:-1], aug_data[:,-1])

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002201 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2040
[LightGBM] [Info] Number of data points in the train set: 30000, number of used features: 8
[LightGBM] [Info] Start training from score 3.403906


LGBMRegressor(max_depth=100)

In [162]:
pred = lgb_rg.predict(X_test[:,:-1])
(pred - X_test[:,-1]).std()
#増殖したデータのみでの予測

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


0.6483851207250447

In [167]:
#さらに増やす
input_z = create_noise(50000, row,)
aug_data = gene(input_z).detach().cpu().numpy()
lgb_rg = lgb.LGBMRegressor(max_depth = 100, n_estimators = 100)
lgb_rg.fit(aug_data[:,:-1], aug_data[:,-1])

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.006388 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2040
[LightGBM] [Info] Number of data points in the train set: 50000, number of used features: 8
[LightGBM] [Info] Start training from score 3.404857


LGBMRegressor(max_depth=100)

In [168]:
pred = lgb_rg.predict(X_test[:,:-1])
(pred - X_test[:,-1]).std()
#増殖したデータのみでの予測

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


0.6511483299173231